In [20]:
import scrapy
from bs4 import BeautifulSoup
import requests
import lxml

In [40]:
url = 'https://www.ign.com/reviews/movies'

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

html_text = requests.get(url, headers=headers).text

In [42]:
soup = BeautifulSoup(html_text, 'lxml')
main = soup.find('main', {"id": 'main-content'})
sections = main.section

In [43]:
print(sections)

<section class="grid jsx-949309242"><section class="box-wrapper jsx-537794787 grid-heading"><div class="jsx-651505178 filter-heading page-content"><h2 class="title1 jsx-4293198083 filter-title">Movie Reviews</h2></div><div class="jsx-1423006382 page-content"><div aria-label="Filters" class="jsx-3963919147 jsx-4243969252 tab-filter-bar jsx-3933161798" data-cy="filter-bar" role="tablist"><a aria-selected="false" class="tab-filter jsx-3710415076" href="/reviews" id="all" role="tab" tabindex="-1"><div class="interface jsx-153568585 bold small">All</div></a><a aria-selected="false" class="tab-filter jsx-3710415076" href="/reviews/games" id="games" role="tab" tabindex="-1"><div class="interface jsx-153568585 bold small">Games</div></a><a aria-selected="true" class="tab-filter jsx-3710415076 active" href="/reviews/movies" id="movies" role="tab" tabindex="0"><div class="interface jsx-153568585 bold small">Movies</div></a><a aria-selected="false" class="tab-filter jsx-3710415076" href="/reviews